The idea here is to cluster neighbourhoods with similar pickup profiles to find similar neighbourhoods

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Data Import
df = pd.read_csv('../data/processed/zones_data_11262015.csv')
df.head()

,pu_time,do_time,pu_nbrhood,do_nbrhood,pass_count,distance,fare,tip,total,payment_type,trip_type
0,11/26/2015 10:17:39 PM,11/26/2015 10:21:03 PM,QN29,QN21,6,0.90,5.0,0.00,6.30,2,1
1,11/26/2015 05:48:01 PM,11/26/2015 05:57:20 PM,MN40,MN17,1,1.94,9.5,0.00,10.30,2,1
2,11/26/2015 06:34:49 PM,11/26/2015 06:48:47 PM,MN09,BX39,1,3.08,12.5,0.00,13.30,2,1
3,11/26/2015 02:25:56 AM,11/26/2015 02:47:10 AM,BK68,BK78,3,3.94,17.0,0.00,18.30,2,1
4,11/26/2015 02:42:24 PM,11/26/2015 02:47:39 PM,BK75,BK61,1,0.50,5.0,1.15,6.95,1,1


In [4]:
pu_group = df.groupby('pu_nbrhood').count()
pu_group.head()

,pu_time,do_time,do_nbrhood,pass_count,distance,fare,tip,total,payment_type,trip_type
pu_nbrhood,,,,,,,,,,
BK09,570,570,570,570,570,570,570,570,570,570
BK17,98,98,98,98,98,98,98,98,98,98
BK19,11,11,11,11,11,11,11,11,11,11
BK21,32,32,32,32,32,32,32,32,32,32
BK23,4,4,4,4,4,4,4,4,4,4


### Data Prep
For the purposes of clustering, I want an hourly profile of pickup numbers per neighbourhood.

What do I need to do:
- get a list of the neighbourhoods
- convert pu times to pandas datetime and isolate hours
- create function to cycle through each hour and each neighbour and create unique column for each hour

In [102]:
# Create list of NYC neighbourhoods
vector_df = df.pu_nbrhood.unique()
vector_df

array(['QN29', 'MN40', 'MN09', 'BK68', 'BK75', 'QN17', 'MN03', 'MN33',
       'QN31', 'BX34', 'MN34', 'QN25', 'MN11', 'BK37', 'QN28', 'BX17',
       'QN68', 'BK90', 'QN22', 'BX40', 'BK61', 'BK69', 'QN70', 'BX36',
       'BK96', 'BK73', 'BK33', 'BK41', 'QN63', 'BK79', 'QN60', 'BK78',
       'MN36', 'MN06', 'MN35', 'XX00', 'BK85', 'QN27', 'BK32', 'QN26',
       'QN06', 'BX05', 'BK81', 'BK76', 'BK88', 'QN72', 'MN04', 'BX39',
       'BK64', 'QN53', 'BX43', 'QN12', 'BX30', 'BK77', 'BX55', 'QN61',
       'BK38', 'BK17', 'BK35', 'BK82', 'QN30', 'BK42', 'QN18', 'BX14',
       'BK09', 'BX63', 'QN01', 'BK40', 'BX26', 'BX28', 'BX01', 'BX27',
       'BX09', 'QN71', 'BX44', 'BK63', 'BX49', 'QN56', 'BX29', 'BK91',
       'BX35', 'QN46', 'QN20', 'MN31', 'BX46', 'BX99', 'BK31', 'BX59',
       'BX33', 'QN50', 'BK95', 'BK60', 'BX06', 'BX37', 'BX41', 'QN08',
       'BK72', 'QN55', 'BK58', 'BX08', 'QN35', 'BK46', 'BK44', 'BK45',
       'QN34', 'SI37', 'QN37', 'BX62', 'QN02', 'BK21', 'BK83', 'QN54',
      

In [103]:
# Find total trips per neighbourhood
count_df = df.groupby('pu_nbrhood').count()
count_df = count_df.loc[:, ['pu_time']]
count_df.columns = ['total']
count_df.head()

,total
pu_nbrhood,
BK09,570
BK17,98
BK19,11
BK21,32
BK23,4


In [104]:
nbrhood_df = df.loc[:,['pu_time', 'pu_nbrhood']]
nbrhood_df['pu_time'] = pd.to_datetime(nbrhood_df['pu_time'])

In [105]:
nbrhood_df['day'] = nbrhood_df['pu_time'].dt.day
nbrhood_df['hour'] = nbrhood_df['pu_time'].dt.hour
nbrhood_df.head()

,pu_time,pu_nbrhood,day,hour
0,2015-11-26 22:17:39,QN29,26,22
1,2015-11-26 17:48:01,MN40,26,17
2,2015-11-26 18:34:49,MN09,26,18
3,2015-11-26 02:25:56,BK68,26,2
4,2015-11-26 14:42:24,BK75,26,14


In [106]:
# df_temp = nbrhood_df[nbrhood_df.hour == 1]
# df_temp = df_temp.groupby('pu_nbrhood').count()

# temp_name = 'hour_{}'.format(1)

# df_temp[temp_name] = df_temp['hour']
# df_temp = df_temp.loc[:, temp_name]
# df_temp = df_temp.to_frame()
# df_temp
# count_df.join(df_temp, how='left').head()

In [107]:
for i in range(1, 25):
    df_temp = nbrhood_df[nbrhood_df.hour == i]
    df_temp = df_temp.groupby('pu_nbrhood').count()

    temp_name = 'hour_{}'.format(i)
    
    df_temp[temp_name] = df_temp['hour']
    df_temp = df_temp.loc[:, temp_name]
    df_temp = df_temp.to_frame()
    count_df = count_df.join(df_temp, how='left')

In [108]:
count_df
count_df

,total,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_24
pu_nbrhood,,,,,,,,,,,,,,,,,,,,,
BK09,570,13.0,9.0,NaN,2.0,5.0,8.0,12.0,21.0,16.0,...,55.0,56.0,25.0,30.0,35.0,34.0,36.0,24.0,27.0,NaN
BK17,98,2.0,1.0,2.0,NaN,NaN,2.0,1.0,3.0,3.0,...,4.0,7.0,5.0,9.0,6.0,6.0,7.0,8.0,4.0,NaN
BK19,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,1.0,1.0,2.0,NaN,NaN
BK21,32,1.0,NaN,1.0,2.0,NaN,NaN,NaN,1.0,NaN,...,2.0,4.0,2.0,3.0,4.0,3.0,4.0,NaN,1.0,NaN
BK23,4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
BK25,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
BK26,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
BK27,5,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
BK28,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,1.0,NaN,6.0,5.0,3.0,NaN,1.0,NaN
